In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

train = pd.read_csv('/kaggle/input/emotion-classification-nlp/emotion-labels-train.csv')
test = pd.read_csv('/kaggle/input/emotion-classification-nlp/emotion-labels-test.csv')
val = pd.read_csv('/kaggle/input/emotion-classification-nlp/emotion-labels-val.csv')

In [ ]:
data = pd.concat([train, test, val], ignore_index=True)

In [ ]:
data.shape

In [ ]:
sia = SentimentIntensityAnalyzer()


In [ ]:
sentiments = []
for text in data['text']:
    sentiment = sia.polarity_scores(text)
    sentiments.append(sentiment)

In [ ]:
sentiment_labels = []
for sentiment in sentiments:
    compound_score = sentiment['compound']
    if compound_score >= 0.05:
        sentiment_labels.append('Positive')
    elif compound_score <= -0.05:
        sentiment_labels.append('Negative')
    else:
        sentiment_labels.append('Neutral')

In [ ]:
# Add the sentiment labels to the DataFrame
data['sentiment'] = sentiment_labels

In [ ]:
data[['text', 'sentiment', 'label']].head(10)


In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
X = data[['text','sentiment']]
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train['sentiment'] = le.fit_transform(X_train['sentiment'])
X_test['sentiment'] = le.fit_transform(X_test['sentiment'])

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train['text'])
X_test_tfidf = vectorizer.transform(X_test['text'])

In [ ]:
model = SVC()
model.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = model.predict(X_test_tfidf)

In [ ]:
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
def predict_sentiment(text):
    text_tfidf = vectorizer.transform([text])
    sentiment = model.predict(text_tfidf)
    return sentiment[0]

In [ ]:
new_review = "Im happy"
predicted_sentiment = predict_sentiment(new_review)
print("Predicted sentiment:", predicted_sentiment)

In [ ]:
new_review = "Im feel confussed "
predicted_sentiment = predict_sentiment(new_review)
print("Predicted sentiment:", predicted_sentiment)

In [ ]:
new_review = "I hate all "
predicted_sentiment = predict_sentiment(new_review)
print("Predicted sentiment:", predicted_sentiment)